Once installed, you need to load the `pytorch_memlab` IPython extensions:

In [1]:
%load_ext pytorch_memlab

One magic is provided, `mlrun` which can act either as a line magic `%mlrun`, or as a cell magic `%%mlrun`

In [2]:
%%mlrun?

First we need some torch code to profile:

In [3]:
import torch

def x():
    torch.nn.Linear(100, 100).cuda()
    
def y(gpu=0):
    torch.nn.Linear(1000, 100).cuda(device=gpu)

We can profile multiple functions at the same type by repeatedly specifying `-f`

In [4]:
%%mlrun -f x -f y

x()
y()

active_bytes,reserved_bytes,line,code
all,all,,
peak,peak,,
0.00B,0.00B,3,def x():
40.00K,2.00M,4,"torch.nn.Linear(100, 100).cuda()"
active_bytes,reserved_bytes,line,code
all,all,,
peak,peak,,
0.00B,2.00M,6,def y(gpu=0):
391.50K,2.00M,7,"torch.nn.Linear(1000, 100).cuda(device=gpu)"


You can alos profile with the `%mlrun` line magic

In [5]:
def z():
    torch.nn.Linear(100, 100).cuda()
%mlrun -f z z()

active_bytes,reserved_bytes,line,code
all,all,,
peak,peak,,
0.00B,0.00B,1,def z():
40.00K,2.00M,2,"torch.nn.Linear(100, 100).cuda()"


You can specify which GPU you wish to profile using `-g`:

In [6]:
%%mlrun -f x -f y -g 0 y

x()
y(gpu=0)

active_bytes,reserved_bytes,line,code
all,all,,
peak,peak,,
0.00B,0.00B,3,def x():
40.00K,2.00M,4,"torch.nn.Linear(100, 100).cuda()"
active_bytes,reserved_bytes,line,code
all,all,,
peak,peak,,
0.00B,2.00M,6,def y(gpu=0):
391.50K,2.00M,7,"torch.nn.Linear(1000, 100).cuda(device=gpu)"


You can get a handle on the `LineProfiler` object using `-r`

In [7]:
profiler = %mlrun -q -r -f x x()
profiler.display()

active_bytes,reserved_bytes,line,code
all,all,,
peak,peak,,
0.00B,0.00B,3,def x():
40.00K,2.00M,4,"torch.nn.Linear(100, 100).cuda()"


You can dump stats out to a file using `-T`:

In [8]:
%mlrun -q -T profile.log -f x x()

active_bytes,reserved_bytes,line,code
all,all,,
peak,peak,,
0.00B,0.00B,3,def x():
40.00K,2.00M,4,"torch.nn.Linear(100, 100).cuda()"


In [9]:
!head profile.log

## x

active_bytes reserved_bytes line  code                                 
         all            all                                            
        peak           peak                                            
       0.00B          0.00B    3  def x():                             
      40.00K          2.00M    4      torch.nn.Linear(100, 100).cuda() 
